In [1]:
# enter at past 10 candles 1h rsi 35 oversold, rsi_length 18, 15m cross k > d, 4h ema1 > ema3
# exit at 1h rsi 74 overbought
# stop loss 2 atr
# symbol ftm

In [2]:
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt
import time
import ta
import os

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [3]:
# 調整 notebook 寬
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% ! important; }<style>"))

# 調整 output 不折疊
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [4]:
import winsound
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [5]:
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)

     asset        free      locked
0      BTC  0.00000000  0.00000000
1      LTC  0.00000000  0.00000000
2      ETH  0.00000000  0.00000000
3      NEO  0.00000000  0.00000000
4      BNB  0.04622450  0.00000000
..     ...         ...         ...
537  WBETH  0.00000000  0.00000000
538    SUI  0.00000000  0.00000000
539   PEPE        0.00        0.00
540  FLOKI        0.00        0.00
541  COMBO  0.00000000  0.00000000

[542 rows x 3 columns]


In [6]:
timezone = 8
symbol_arr = ['imxusdt']
interval = '1h'

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 3600
start_time = round(time.time() * 1000 - (86400 * 1000 * 10))
end_time = round(time.time() * 1000)

def set_time():
    
    global start_time, end_time
    
    # start epoch till now, use prior 5 days for this strategy
    start_time = round(time.time() * 1000 - (86400 * 1000 * 10))
    end_time = round(time.time() * 1000)


In [7]:
dataframes = {}
def create_raw(symbol_arr, interval_arr, step):
    global start_time, end_time
    
    for interval in interval_arr:
        for symbol in symbol_arr:
            set_time()
            # Fetch the data using batch requests
            data = []
            while start_time < end_time:
                limit = min(step, end_time - start_time + 1)  # Adjust the limit for the last batch
                response = client.get_klines(symbol=symbol.upper(), interval=interval, limit=limit, startTime=start_time)

                if len(response) == 0:
                    break  # No more data available, exit the loop
                data.extend(response)
                start_time = response[-1][0] + 1

            # Convert the data to a DataFrame
            columns = [
                "Open_Time", "Open", "High", "Low", "Close", "Volume", "Close_Time",
                "Quote asset volume", "Number of trades", "Taker buy base asset volume",
                "Taker buy quote asset volume", "Ignore"
            ]
            raw_df = pd.DataFrame(data, columns=columns)   

            raw_df = raw_df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']]

            dataframes[f'df_{interval}_{symbol}'] = raw_df

In [8]:
interval_arr = ['1h', '4h']
ema_arr = [8, 18, 38]

In [9]:
ep_per = 1.003
sl_per = 0.998
sl_det = 'Close'

In [10]:
def get_klines(symbol, interval):
    global dataframes
    df = dataframes[f'df_{interval}_{symbol}']
    df = df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)
    df = df.set_index('Open_Time')

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
    df = df[~df.index.duplicated(keep='first')]

    return df

In [11]:
loop_start_time = time.time()
create_raw(symbol_arr, interval_arr, step)
loop_end_time = time.time()
print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")
print(dataframes)

Time taken to execute for loop: 0.33300352096557617 seconds
{'df_1h_imxusdt':          Open_Time        Open       Close        High         Low           Volume
0    1685754000000  0.81400000  0.80700000  0.81900000  0.80500000  411295.63000000
1    1685757600000  0.80700000  0.81100000  0.81300000  0.80400000  135100.66000000
2    1685761200000  0.81100000  0.81800000  0.82000000  0.81000000  215265.69000000
3    1685764800000  0.81800000  0.81800000  0.82000000  0.80900000  365396.52000000
4    1685768400000  0.81900000  0.80900000  0.83000000  0.80600000  365885.46000000
..             ...         ...         ...         ...         ...              ...
235  1686600000000  0.61500000  0.62000000  0.62100000  0.61400000   95062.14000000
236  1686603600000  0.62000000  0.62300000  0.62400000  0.61900000   68396.59000000
237  1686607200000  0.62300000  0.62000000  0.62600000  0.61900000  202333.30000000
238  1686610800000  0.62100000  0.62100000  0.62500000  0.61800000  116371.3700000

In [12]:
symbol_dfs = {}
def multi_timeframes(symbol):
    df = get_klines(symbol, interval_arr[0]).copy() # eth 1h
    df = df.astype(float).round(4)

    for interval in interval_arr:
        raw_df = get_klines(symbol, interval)
        # ema
        for ema in ema_arr:
            column_name = f'ema_{ema}_{interval}' # eth 1h 4th ema
            df[column_name] = ta.trend.EMAIndicator(raw_df.Close, window=ema, fillna=True).ema_indicator()

        # rsi
    #     rsi = ta.momentum.RSIIndicator(raw_df.Close, window = rsi_int)
    #     df[f'rsi_{interval}'] = rsi.rsi()

        # atr
    #         df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)


    # reset index and set current index as a column
    df = df.reset_index()

    # set new index with integers
    df = df.set_index(pd.RangeIndex(len(df)))

    # time_format(timezone)
    df['Open_Time'] = df['Open_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # fill up higher time frame empty values with equal interval between each value
    df = df.replace('', np.nan)
    df['ema_8_4h'] = df['ema_8_4h'].interpolate()
    df['ema_18_4h'] = df['ema_18_4h'].interpolate()
    df['ema_38_4h'] = df['ema_38_4h'].interpolate()
    symbol_dfs[f'df_{symbol}'] = df
    
    return symbol_dfs[f'df_{symbol}']

In [13]:
# def check_cross(df, kd_dir):
#     up = df['slow_k'] > df['slow_d']
#     down = df['slow_k'] < df['slow_d']
#     if kd_dir == 'Up':
#         return up.diff() & up
#     if kd_dir == 'Any':
#         return up.diff()
#     if kd_dir == 'Down':
#         return down.diff() & down


# def indicators(df, kd_dir):
def indicators(df):    
    
# bb    
    bb_int = 30
    bb_dev = 2
    bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
    df['bb_u'] = bb.bollinger_hband()
    df['bb_m'] = bb.bollinger_mavg()
    df['bb_l'] = bb.bollinger_lband()  
    
# kd
#     df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], 14, 3)
#     df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], 14, 3)
    
# kd cross
#     df['kd_cross'] = check_cross(df, kd_dir)

In [14]:
def conditions(df):
    
    df['c1'] = df['ema_8_1h'] >= df['ema_18_1h']
    df['c2'] = df['ema_18_1h'] >= df['ema_38_1h']
    df['c3'] = df['ema_8_4h'] >= df['ema_38_4h']

    # 條件達成
    df['signal'] = df.c1 & df.c2 & df.c3
            
    # 下一根進場
    df['entry'] = False

In [15]:
def cancel_orders(symbol):
    
    open_orders = client.futures_get_open_orders()
    
    # Cancel all open orders
    if open_orders:
        # Cancel all open orders
        for order in open_orders:
            if (order['symbol'] == symbol.upper()) & (order['positionSide'] == 'LONG'):  
                cancel_response = client.futures_cancel_order(symbol=symbol.upper(), orderId=order['orderId'])
                print(f"Canceled order: {order['symbol']} - {order['orderId']}")
    else:
        # Continue with the rest of the code
        print("No open LONG orders found.")
        time.sleep(1)  # Sleep for 1 second to avoid API rate limit

In [16]:
# check current price, if hits ema_18_4h, buy, one candle only enter once

In [17]:
def enter_position(df, symbol):
        
    #-----Calculate entry price-----#
    open_val = df['Open']
    close_val = df['Close']

    ema_18_val = df['ema_18_4h']
    
    df.loc[df.index[-1], 'entry_p'] = ema_18_val.loc[close_val.index[-1]] * ep_per

    #-----position attributes-----#
    usdt_q = 20
    quantity = usdt_q / df.loc[df.index[-1], 'Close']
    entry_p = round(df.loc[df.index[-1], 'entry_p'], 4)
#     stop_loss_p = round(df.loc[df.index[-1], 'stop_loss'], 4)
#     take_profit_p = round(df.loc[df.index[-1], 'take_profit'], 2)
    
    cancel_orders(symbol)

    try:
        order = client.futures_create_order(
            symbol=symbol.upper(),
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='LONG',
        )
        print('Order created successfully.')       
            
    except Exception as e:
        print(f'Error creating order: {e}')
        
    print(str(symbol.upper()) + ' entered at ' + str(entry_p))

    return df

In [18]:
def check_sl(df, symbol, current_k):
    
    # 檢查前一根 Close 是否低於 SL，若低於，即刻停損
    
    last_signal_row = df.loc[df['signal'].eq(True) & df['Open_Time'].ne(current_k)].tail(1)
    if not last_signal_row.empty:
        current_sl = round(df.loc[last_signal_row.index[0], 'Low'] * sl_per, 4)
#         current_tp = round(df.loc[last_signal_row.index[0], 'Close'] + df.loc[last_signal_row.index[0], 'atr'] * tp_atr, 4)

    print(f'Current SL at {current_sl}')
    print(f'Current TP at DYNAMIC')
    
    # 確定停損
    if (df.iloc[df.index[-2]]['Low'] <= current_sl):
        
        print('Last candle closed below SL, try create SL Order')
        
        cancel_orders(symbol)

        # 如果目前價格高於停損
        ticker = client.futures_symbol_ticker(symbol=symbol.upper())

        mark_price = round(float(ticker['price']), 4)
        
        stop_price = current_sl
        if current_sl > mark_price:
            stop_price = mark_price
        
        try:

            stop_loss_order = client.futures_create_order(
                symbol=symbol.upper(),
                side='SELL',
                type='STOP_MARKET',
                stopPrice=stop_price,
                stopLimitTimeInForce='GTC',
                closePosition = 'true',
                positionSide = 'LONG'
            )
        
            print(f'Stop loss created successfully at: {current_sl}')
            winsound.Beep(frequency, duration)
        
        except Exception as e:
            
            print(f'Error creating SL order: {e}')
          

In [19]:
def check_tp(df, symbol):
    
    try:
        # 隨時偵測出場條件是否成立，不必等 Close 發生
        if ((df.loc[df.index[-1], 'Close'] >= df.loc[df.index[-1], 'bb_u']) or 
            ((df.loc[df.index[-2], 'Volume'] > df.loc[df.index[-3], 'Volume']) & # vol greater
            (df.loc[df.index[-3], 'Open'] < df.loc[df.index[-3], 'Close']) & # previous green candle
            (df.loc[df.index[-2], 'Open'] > df.loc[df.index[-2], 'Close']) & # current red candle
            (df.loc[df.index[-2], 'Low'] < df.loc[df.index[-3], 'Open']))
           ): # current low lower than previous open
                
            take_profit_order = client.futures_create_order(
                symbol=symbol.upper(),
                side='SELL',
                type='TAKE_PROFIT_MARKET',
                stopPrice=df.loc[df.index[-1], 'Close'],
                stopLimitTimeInForce='GTC',
                closePosition = 'true',
                positionSide = 'LONG'
            )
            winsound.Beep(frequency, duration)
            print('Take profit created successfully')
            
        else:
            print(f"Take profit target not reached bb_u:{round(df.loc[df.index[-1], 'bb_u'], 4)}")
            
    except Exception as e:
        print(f'Error creating TP order: {e}')

In [20]:
import IPython

def restart_and_run_all():
    # Restart the kernel
    IPython.Application.instance().kernel.do_shutdown(True)

    # Re-run all code cells
    IPython.get_ipython().run_line_magic('run', '-i 000_023_LONG_1h_ema_4h_ema_imxusdt.ipynb')


In [21]:
from IPython.display import clear_output


def console_log(df, symbol):
    try:
        df = df.reset_index(drop=True)
        df = df.round(4)
        print('----------------------------------------------------------------------------')
        print(f"{symbol} - {str(df.loc[df.index[-1], 'Open_Time'])} at {str(df.loc[df.index[-1], 'Close'])}")
        print()
        print(df[['Open_Time', 'Open', 'Low', 'Close', 'Volume', 'c1', 'c2', 'c3', 'bb_u', 'ema_18_4h', 'signal', 'entry']].tail(5))
        print('----------------------------------------------------------------------------')
        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Side':position['positionSide'],
                                            'Entry_P':round(float(position['entryPrice']),4),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],2),
                                            'PL':round(float(position['unrealizedProfit']),2),
                                            'X':round(float(position['leverage']),1),
                                           }, index=[0])      
                print(position_df)


    except Exception as e:
        print(f'Error UPDATING info: {e}')    


In [22]:
current_bar_pos = False
def check_price(df, symbol):
    global current_k, current_bar_pos
    
    try:
        if ((df.iloc[df.index[-1]]['signal']) & # 三線條件成立
            (df.iloc[df.index[-1]]['Close'] <= df.iloc[df.index[-1]]['ema_18_4h'] * ep_per) & # 價格回落
            (current_bar_pos == False)): # 本 K 未進場
            enter_position(df, symbol)
            winsound.Beep(frequency, duration)
            df.loc[df.index[-1], 'entry'] = True
            current_bar_pos = True
            
        # 如果時間標籤改變，設 current_bar_pos 為可進場
        if df['Open_Time'][len(df) - 1] != current_k:

            print('time changed')
            current_k = df['Open_Time'][len(df) - 1]
            current_bar_pos = False
            
    except Exception as e:
        print(f'Error creating ENTRY order: {e}')

In [23]:
loop_start_time = 0
loop_end_time = 0

line_count = 0
max_lines = 1
current_k = 0
restart_countdown = 2 * 60 * 60 # restart in 2 hours

def run():
    global current_k, line_count, max_lines, restart_countdown, loop_start_time, loop_end_time

    while True:
        loop_start_time = time.time()
        console_df = {}
        create_raw(symbol_arr, interval_arr, step)
        try:
            set_time()
            for symbol in symbol_arr:
                df = multi_timeframes(symbol)
#                 indicators(df, kd_dir)
                indicators(df)
                conditions(df)
                check_price(df, symbol)
                console_df[symbol] = df
                # 更新狀態
                console_log(console_df[symbol], symbol)    

                # 檢查停損
                try:
                    positions_info = client.futures_account()['positions']
                    long_positions = [p for p in positions_info if p['positionSide'] == 'LONG' and float(p['positionAmt']) != 0 and p['symbol'] == symbol.upper()]

                    if long_positions:
                        check_tp(df, symbol)
                        check_sl(df, symbol, current_k)

                    else:
                        print(f'No LONG position. No SL for {symbol}')

                except Exception as e:
                    print(f'Error checking SL for {symbol}: {e}')
          
        except ConnectionError as e:
            print("Connection error occurred:", e)
            print("Retrying in 5 seconds...")
            time.sleep(5)
        
        print('----------------------------------------------------------------------------')
            
        # countdown for restarting session
        restart_countdown -= 1
        print(f'{restart_countdown} iterations until restart')

        if restart_countdown <= 100:
            restart_and_run_all()

        # count each loop time
        loop_end_time = time.time()
        print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")            

        line_count += 1
        if line_count >= max_lines:
            clear_output(wait=True)
            line_count = 0    
            
        time.sleep(3)

In [24]:
# c1 ema 8 > ema 18
# c2 ema 18 > ema 38
# c3 ema 8 4h > ema 38 4h
# exit at 1h close > bb_u, or bearish engulf + volume, or low < previous open

# exiting conditions
# high greater than bb_u 
# red candle engulf + volume previous greend candle and red candle low lower than previous green candle open

In [25]:
run()   

ReadTimeout: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out. (read timeout=10)